In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score

pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


In [3]:
np.random.random()

0.7098987032239765

# Load data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)

poem_lines = []
with open('/content/gdrive/MyDrive/Colab Notebooks/lazyprogrammer/data/robert_frost.txt', 'r') as file:
  for line in file:
    txt = line.rstrip().lower()
    if txt:
      txt = txt.translate(str.maketrans('', '', string.punctuation))
      poem_lines.append(txt)

poem_lines = np.array(poem_lines)
print(poem_lines[:5])

Mounted at /content/gdrive/
['two roads diverged in a yellow wood' 'and sorry i could not travel both'
 'and be one traveler long i stood'
 'and looked down one as far as i could'
 'to where it bent in the undergrowth']


In [ ]:
X_train = poem_lines

# Utils

In [ ]:
END_TOKEN = '<END>'

def build_word2idx(txt_lines, skip_word_filter=None, end_token=None):
  word2idx = {'<unknown>': 0}
  if end_token is not None:
    word2idx = {end_token: 1}
  idx = len(word2idx)
  for line in txt_lines:
    for token in line.split():
      if token in word2idx:
        continue
      elif skip_word_filter is not None and not skip_word_filter(token):
        continue
      else:
        word2idx[token] = idx
        idx += 1
  return word2idx

def tokenize(txt, word2idx, end_token=None):
  vector = []
  for token in txt.split():
    vector.append(word2idx.get(token, 0))
  if end_token is not None:
    vector.append(word2idx[end_token])
  return vector

def idx2word(token_idxs, word2idx):
  idx2token = {}
  for k, v in word2idx.items():
    idx2token[v] = k

  return [idx2token[idx] for _, idx in enumerate(token_idxs)]


## Markov Model class

In [ ]:
class SparseSecondOrderMarkovModel():
  def __init__(self, n_states, seq_end=None):
    self.n_states = n_states
    self.pi = None
    self.A1 = None
    self.A2 = None
    self.seq_end = seq_end
  
  def fit(self, x):
    assert len(x) > 0

    # list all states from x at every sequence position t
    x_pi = [] # list(state_0)
    x_A1 = {} # dict(state_0, list(state_1))
    x_A2 = {} # dict((state_t_2, state_t_1), list(state_t))
    for idx, x_cur in enumerate(x):
      state_0 = x_cur[0]
      x_pi.append(state_0)
      x_A1[state_0] = x_A1.get(state_0, [])
      if len(x_cur) == 1:
        if self.seq_end is not None:
          x_A1[state_0].append(self.seq_end)
      else:
        state_1 = x_cur[1]
        x_A1[state_0].append(state_1)
        for t in range(2, len(x_cur)):
          pair_t_1 = (x_cur[t-2], x_cur[t-1])
          x_A2[pair_t_1] = x_A2.get(pair_t_1, [])
          x_A2[pair_t_1].append(x_cur[t])

    # count the probability of each transition from dictionaries above,
    # this will give us sparse matrix (implemented as dictionary) of transition states
    self.pi = self._calc_discrete_distribution(x_pi) # dict(state_0, state_0 probability at first position)
    
    self.A1 = {} # dict(state_0, dict(state_1, state_0 -> state_1 transition probability at first two positions position))
    for state_1, cur_state in x_A1.items():
      self.A1[state_1] = self._calc_discrete_distribution(cur_state)
        
    self.A2 = {} # dict((state_t_2, state_t_1), dict(state_t, (state_t_2, state_t_1) -> state_t  transition probability in the whole sequence))
    for state_1, cur_state in x_A2.items():
      self.A2[state_1] = self._calc_discrete_distribution(cur_state)

  def _calc_discrete_distribution(self, tokens: list):
    n = len(tokens)
    pdf = {}
    for i, token in enumerate(tokens):
      pdf[token] = pdf.get(token, 0) + 1
    
    for k, val in pdf.items():
      pdf[k] = val / n   
    return pdf


class MAPDiscreteSequenceGenerator(): # Maximum Posteriori
  def __init__(self, likelihood_model):
    self.model = likelihood_model

  def generate(self, max_steps):
    assert max_steps > 0
    x_generated = []
    
    # generate token 0
    token_0_distribution = self.model.pi
    x_generated.append(self._cdf_inv(token_0_distribution))

    # generate token 1
    if max_steps > 1:
      token_1_distribution = self.model.A1[x_generated[0]]
      next_token = self._cdf_inv(token_1_distribution)
      x_generated.append(self._cdf_inv(token_1_distribution))
      if self.model.seq_end is not None and next_token == self.model.seq_end:
        return x_generated

    # generate tokens >= 2
    for t in range(2, max_steps):
      token_t_distribution = self.model.A2[(x_generated[-2], x_generated[-1])]
      next_token = self._cdf_inv(token_t_distribution)
      x_generated.append(next_token)
      if self.model.seq_end is not None and next_token == self.model.seq_end:
        break
    
    return x_generated
  

  def _cdf_inv(self, discrete_distribution: dict):
    u = np.random.random()
    cdf = 0.0
    random_token = None
    for token, prob in discrete_distribution.items():
      cdf += prob
      if u < cdf:
        return token
    raise Exception('Unexpected line execution. Probably provided discrete distribution was not correct')

# Processing

In [ ]:
word2idx_train = build_word2idx(X_train, end_token=END_TOKEN)
V = len(word2idx_train)
print('vocab size V:', V)

X_train_vectorized = np.array([tokenize(line, word2idx_train, end_token=END_TOKEN) for line in X_train], dtype=object)
X_train_vectorized[:3]

vocab size V: 2198


array([list([1, 2, 3, 4, 5, 6, 7, 1]),
       list([8, 9, 10, 11, 12, 13, 14, 1]),
       list([8, 15, 16, 17, 18, 10, 19, 1])], dtype=object)

In [ ]:
seq_model = SparseSecondOrderMarkovModel(V, seq_end=word2idx_train[END_TOKEN])
seq_model.fit(X_train_vectorized)
generator = MAPDiscreteSequenceGenerator(seq_model)

In [ ]:
print(' '.join(idx2word(generator.generate(3), word2idx_train)))
print(' '.join(idx2word(generator.generate(5), word2idx_train)))
print(' '.join(idx2word(generator.generate(10), word2idx_train)))
print(' '.join(idx2word(generator.generate(30), word2idx_train)))

the darkest evening
and sometimes something seemed to
a moment he was two
up attic mother two
